In [48]:
import re
import pandas as pd
from ydata_profiling import ProfileReport


In [49]:
df=pd.read_csv("Data.csv")
df.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


In [50]:
p=ProfileReport(df)
p.to_file(output_file='output.html')

Summarize dataset:  81%|████████▏ | 22/27 [00:00<00:00, 70.24it/s, Calculate auto correlation]          /home/e02964/.local/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'Function <code object pandas_auto_compute at 0x746550f06600, file "/home/e02964/.local/lib/python3.10/site-packages/ydata_profiling/model/pandas/correlations_pandas.py", line 167>')
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 60.58it/s]


In [51]:
def extract(value):
    match = re.match(r'^\d+', value)
    return match.group(0)
def extract_rpm(value):
    numbers = re.findall(r'\d+', value)
    numbers = list(map(int, numbers))
    if len(numbers)<2:
        return 4000
    return numbers[1]

In [52]:
counts=df['Make'].value_counts()
other=counts[counts<=100].index
df['Make']=df['Make'].replace(other,'Other')

In [53]:
df=df.dropna(subset=['Engine'])
df['Drivetrain'] = df['Drivetrain'].fillna(df['Drivetrain'].mode())
df['Fuel Tank Capacity']=df['Fuel Tank Capacity'].fillna(df['Fuel Tank Capacity'].mean())
df['Engine']=df['Engine'].apply(lambda x :x[:-2]).astype(float)
df['Power']=df['Max Power'].apply(extract)
df['Torque']=df['Max Torque'].apply(extract)
df['Power_Rpm']=df['Max Power'].apply(extract_rpm)
df['Torque_Rpm']=df['Max Torque'].apply(extract_rpm)

/tmp/ipykernel_106504/287624946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Drivetrain'] = df['Drivetrain'].fillna(df['Drivetrain'].mode())
/tmp/ipykernel_106504/287624946.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fuel Tank Capacity']=df['Fuel Tank Capacity'].fillna(df['Fuel Tank Capacity'].mean())
/tmp/ipykernel_106504/287624946.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [54]:
df.to_csv("Cleaned_Data.csv",index=False)